In [2]:
from pyswarm import pso
import pandas as pd
import numpy as np

# 从Excel文件中读取数据
xls = pd.ExcelFile("E:\\数学建模国赛2023选题\\C题\\第二题建模约束参数.xlsx")
model_params_df = pd.read_excel(xls, 'Model_Params')
markup_bounds_df = pd.read_excel(xls, 'Markup_Bounds')
sales_bounds_df = pd.read_excel(xls, 'Sales_Bounds')
daily_sales_bounds_df = pd.read_excel(xls, 'Daily_Sales_Bounds')

markup_lower_bound = markup_bounds_df['lower_bound'].values
markup_upper_bound = markup_bounds_df['upper_bound'].values
sales_lower_bound = sales_bounds_df['lower_bound'].values
sales_upper_bound = sales_bounds_df['upper_bound'].values
daily_sales_lower_bound = daily_sales_bounds_df['lower_bound'].values[0]
daily_sales_upper_bound = daily_sales_bounds_df['upper_bound'].values[0]

# 定义优化函数
def optimize_for_day(day, a_values, b_values):
    def objective(vars):
        x = vars[:6]
        y = vars[6:]
        return -np.sum(x * y)

    def constraints(vars):
        cons = []
        epsilon = 0.00001  # 小的epsilon值用于软约束

        for i in range(6):
            cons.append(abs(vars[6+i] - a_values[i] * np.exp(b_values[i] * vars[i])) - epsilon)

        for i in range(6):
            cons.append(vars[i] - markup_lower_bound[i])
            cons.append(markup_upper_bound[i] - vars[i])
            cons.append(vars[6+i] - sales_lower_bound[i])
            cons.append(sales_upper_bound[i] - vars[6+i])

        cons.append(daily_sales_upper_bound - np.sum(vars[6:]))
        cons.append(np.sum(vars[6:]) - daily_sales_lower_bound)

        return cons

    lb = list(markup_lower_bound) + list(sales_lower_bound)
    ub = list(markup_upper_bound) + list(sales_upper_bound)
    xopt, fopt = pso(objective, lb, ub, f_ieqcons=constraints)
    return xopt, -fopt

# 执行敏感性分析
def sensitivity_analysis(perturbation_levels, base_a_values, base_b_values, days_of_week):
    sensitivity_results = {}
    baseline_results = {}
    for day in days_of_week:
        xopt, fopt = optimize_for_day(day, base_a_values, base_b_values)
        baseline_results[day] = {
            'Optimized Variables': xopt,
            'Optimized Objective': fopt
        }

    for level in perturbation_levels:
        perturbed_results = {}
        perturbed_a_values = base_a_values * (1 + level)
        perturbed_b_values = base_b_values * (1 + level)

        for day in days_of_week:
            xopt, fopt = optimize_for_day(day, perturbed_a_values, perturbed_b_values)
            perturbed_results[day] = {
                'Optimized Variables': xopt,
                'Optimized Objective': fopt
            }

        sensitivity_results[f"Perturbation {level*100}%"] = perturbed_results

    return baseline_results, sensitivity_results

# 天数和扰动水平
days_of_week = ['星期一', '星期二', '星期三', '星期四', '星期五', '星期六', '星期日']
perturbation_levels = [0.05, 0.1, 0.15]

# 执行敏感性分析
baseline_results, sensitivity_results = sensitivity_analysis(
    perturbation_levels,
    model_params_df['a'].values,
    model_params_df['b'].values,
    days_of_week
)

print("基线结果：", baseline_results)
print("敏感性分析结果：", sensitivity_results)


Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping sea

In [19]:
# 敏感性分析结果的数据
sensitivity_data = {
'Perturbation 5.0%': {
        '星期一': 69324.8085281223, '星期二': 70950.92255456524, '星期三': 71626.06348412506,
        '星期四': 72252.3112141553, '星期五': 72232.06841552508, '星期六': 70817.97106105555,
        '星期日': 71705.7657326196
    },
    'Perturbation 10.0%': {
        '星期一': 71910.13636930278, '星期二': 71472.62420502446, '星期三': 72342.53363249302,
        '星期四': 72244.03680631417, '星期五': 70970.1718927859, '星期六': 71293.94076373486,
        '星期日': 72050.86850825211
    },
    'Perturbation 15.0%': {
        '星期一': 71099.11907288048, '星期二': 72041.05614888841, '星期三': 72246.3926171707,
        '星期四': 70552.13102897897, '星期五': 72282.78598558014, '星期六': 71177.54666831797,
        '星期日': 72302.57610972633
    }
}

# 基线结果的数据
baseline_data = {
    '星期一': 70491.33869778173, '星期二': 71902.73513067004, '星期三': 70799.6707028168,
    '星期四': 71748.77082806216, '星期五': 72231.58505096659, '星期六': 71253.92116317408,
    '星期日': 71949.75954144009
}

# 计算每天在10%和15%扰动下的绝对和相对变化
change_analysis_5_10_15 = {}

for day in days_of_week:
    change_analysis_5_10_15[day] = {
        'Absolute Change 5%': abs(sensitivity_data['Perturbation 5.0%'][day] - baseline_data[day]),
        'Relative Change 5%': abs(sensitivity_data['Perturbation 5.0%'][day] - baseline_data[day]) / baseline_data[day] * 100,
        'Absolute Change 10%': abs(sensitivity_data['Perturbation 10.0%'][day] - baseline_data[day]),
        'Relative Change 10%': abs(sensitivity_data['Perturbation 10.0%'][day] - baseline_data[day]) / baseline_data[day] * 100,
        'Absolute Change 15%': abs(sensitivity_data['Perturbation 15.0%'][day] - baseline_data[day]),
        'Relative Change 15%': abs(sensitivity_data['Perturbation 15.0%'][day] - baseline_data[day]) / baseline_data[day] * 100
    }

change_analysis_5_10_15_df = pd.DataFrame(change_analysis_5_10_15).T
change_analysis_5_10_15_df


,Absolute Change 5%,Relative Change 5%,Absolute Change 10%,Relative Change 10%,Absolute Change 15%,Relative Change 15%
星期一,1166.530170,1.654856,1418.797672,2.012726,607.780375,0.862206
星期二,951.812576,1.323750,430.110926,0.598184,138.321018,0.192372
星期三,826.392781,1.167227,1542.862930,2.179195,1446.721914,2.043402
星期四,503.540386,0.701810,495.265978,0.690278,1196.639799,1.667819
星期五,0.483365,0.000669,1261.413158,1.746346,51.200935,0.070884
星期六,435.950102,0.611826,40.019601,0.056165,76.374495,0.107186
星期日,243.993809,0.339117,101.108967,0.140527,352.816568,0.490365


In [ ]:
"""
根据计算得出的绝对和相对变化，我们可以得出以下几点观察：

绝对变化量
对于“星期一”，最大的绝对变化为1166.53，表明该日的"Optimized Objective"对输入参数最敏感。
对于“星期五”，最小的绝对变化仅为0.48，表明该日的"Optimized Objective"对输入参数相对不敏感。
相对变化量（以百分比表示）
对于“星期一”，相对变化为1.65%，也是所有天里最高的。
对于“星期五”，相对变化几乎为零（0.0000067%），这可能表明模型在这一天对参数非常稳健。
综合分析
在5%的扰动下，“星期一”表现出最高的敏感性，而“星期五”则相对稳健。
看到这样的结果，决策者可能需要更加关注“星期一”的业务，因为该日的最优解更容易受到输入参数的影响。


从这些数据中，我们可以看到在10%和15%的扰动下，目标函数的变化相对较小。这意味着模型对这些扰动相对不敏感，表现出较好的稳健性
"""